This is to figure out the total disturbed area over CONUS

In [1]:
import os
import numpy as np
from pathlib import Path
import glob
import pandas
import plotly.express as px
import plotly.graph_objs as go
from sklearn.linear_model import TheilSenRegressor
from scipy import stats
estimator = TheilSenRegressor(random_state=42)
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
from utils import get_area_adjust_factor

years_selected = np.arange(0, 35) * 1 + 1988

years_xsticks = [1988, 1990, 1995, 2000, 2005, 2010, 2015, 2020, 2022]

# see the page to check out how to calcualte the uncertainty
# https://stackoverflow.com/questions/67737684/plotting-confidence-interval-for-linear-regression-line-of-a-pandas-time-series


In [ ]:
## report the percentage of the area at least once disturbance
des = '/gpfs/sharedfs1/zhulab/Shi/ProjectCONUSDisturbanceAgent/Analysis/V08PostRelease/conus_disturbance_once_muti_area'
data_list = sorted(glob.glob(os.path.join(des, "h*"))) # the data was produced by countTileDisturbanceOnce.m 1988-2022
distdata_once = []
for data_file in data_list:
    distdata_once.append(pandas.read_csv(data_file))
distdata_once = pandas.concat(distdata_once)


prt_dist_once = distdata_once['oncedistarea'].sum()/distdata_once['totalarea'].sum()
prt_dist_mutil = distdata_once['mutidistarea'].sum()/distdata_once['totalarea'].sum()
# print total area
print('The total once disturbed area is: ')
print(distdata_once['oncedistarea'].sum())
# print total area
print('The total land area is: ')
print(distdata_once['totalarea'].sum())

print('The percentage of the area at least once disturbance is: ')
print(100*prt_dist_once)

print('The percentage of the area at mutiple disturbance is: ')
print(100*prt_dist_mutil)

The total once disturbed area is: 


KeyError: 'distarea'

In [11]:
## load disturbance dataset
des         = '/gpfs/sharedfs1/zhulab/Shi/ProjectCONUSDisturbanceAgent/Analysis/V08PostRelease/tile_disturbance_measurement' #
data_list = sorted(glob.glob(os.path.join(des, "h*")))
distdata = []
for data_file in data_list:
    distdata.append(pandas.read_csv(data_file))
distdata = pandas.concat(distdata)

# rename to the agents that we want to use
distdata['agent'] = distdata['agent'].replace({'forest_management': 'Logging', 'construction':'Construction', 'agriculture_activity':'Agricultural disturbance', 
                                               'stress':'Stress', 'natural_hazard':'Wind/geohazard', 
                                               'water_dynamic':'Water disturbance', 'fire':'Fire',
                                               'other':'Other'})

print('Level-2 agents in the dataset:')
print(distdata['agent'].unique())

Level-2 agents in the dataset:
['Logging' 'Construction' 'Stress' 'Wind/geohazard' 'Water disturbance'
 'Fire' 'Agricultural disturbance' 'Other']


In [12]:
## manage data into different subregions
years = distdata.year.unique()
agents = distdata.agent.unique()
rec_agent_area_year = []
nca1_agent_area_year = []
nca2_agent_area_year = []
nca3_agent_area_year = []
nca4_agent_area_year = []
nca5_agent_area_year = []
nca6_agent_area_year = []
nca7_agent_area_year = []
for yr in years_selected:
    for agt in agents:
        distdata_year = distdata[distdata.year == yr]
        distdata_agent = distdata_year[distdata_year.agent == agt]
        rec_agent_area_year.append(
            {
                "area": distdata_agent.loc[:, ["area"]].sum(axis=0).area,
                "agent": agt,
                "year": yr,
            }
        )

        nca1_agent_area_year.append(
            {
                "area": distdata_agent.loc[:, ["nca1"]].sum(axis=0).nca1,
                "agent": agt,
                "year": yr,
            }
        )
        nca2_agent_area_year.append(
            {
                "area": distdata_agent.loc[:, ["nca2"]].sum(axis=0).nca2,
                "agent": agt,
                "year": yr,
            }
        )
        nca3_agent_area_year.append(
            {
                "area": distdata_agent.loc[:, ["nca3"]].sum(axis=0).nca3,
                "agent": agt,
                "year": yr,
            }
        )
        nca4_agent_area_year.append(
            {
                "area": distdata_agent.loc[:, ["nca4"]].sum(axis=0).nca4,
                "agent": agt,
                "year": yr,
            }
        )
        nca5_agent_area_year.append(
            {
                "area": distdata_agent.loc[:, ["nca5"]].sum(axis=0).nca5,
                "agent": agt,
                "year": yr,
            }
        )
        nca6_agent_area_year.append(
            {
                "area": distdata_agent.loc[:, ["nca6"]].sum(axis=0).nca6,
                "agent": agt,
                "year": yr,
            }
        )
        nca7_agent_area_year.append(
            {
                "area": distdata_agent.loc[:, ["nca7"]].sum(axis=0).nca7,
                "agent": agt,
                "year": yr,
            }
        )

rec_agent_area_year = pandas.DataFrame(rec_agent_area_year)
nca1_agent_area_year = pandas.DataFrame(nca1_agent_area_year)
nca2_agent_area_year = pandas.DataFrame(nca2_agent_area_year)
nca3_agent_area_year = pandas.DataFrame(nca3_agent_area_year)
nca4_agent_area_year = pandas.DataFrame(nca4_agent_area_year)
nca5_agent_area_year = pandas.DataFrame(nca5_agent_area_year)
nca6_agent_area_year = pandas.DataFrame(nca6_agent_area_year)
nca7_agent_area_year = pandas.DataFrame(nca7_agent_area_year)

In [13]:
# to have level-1
for rec in [rec_agent_area_year, nca1_agent_area_year, nca2_agent_area_year, nca3_agent_area_year, nca4_agent_area_year, nca5_agent_area_year, nca6_agent_area_year, nca7_agent_area_year]:
    rec['agent_level1'] = rec['agent']
    rec.loc[(rec['agent'] == 'Logging'), 'agent_level1'] = 'Human-directed'
    rec.loc[(rec['agent'] == 'Construction'), 'agent_level1'] = 'Human-directed'
    rec.loc[(rec['agent'] == 'Agricultural disturbance'), 'agent_level1'] = 'Human-directed'
    rec.loc[(rec['agent'] == 'Stress'), 'agent_level1'] = 'Wild'
    rec.loc[(rec['agent'] == 'Wind/geohazard'), 'agent_level1'] = 'Wild'
    rec.loc[(rec['agent'] == 'Water disturbance'), 'agent_level1'] = 'Mixed'
    rec.loc[(rec['agent'] == 'Fire'), 'agent_level1'] = 'Wild' # natural means "wild"
    # display the unique agents
    print('Level-1 agents in the dataset:')
    print(rec['agent_level1'].unique())
    print('')
    

Level-1 agents in the dataset:
['Human-directed' 'Wild' 'Mixed' 'Other']

Level-1 agents in the dataset:
['Human-directed' 'Wild' 'Mixed' 'Other']

Level-1 agents in the dataset:
['Human-directed' 'Wild' 'Mixed' 'Other']

Level-1 agents in the dataset:
['Human-directed' 'Wild' 'Mixed' 'Other']

Level-1 agents in the dataset:
['Human-directed' 'Wild' 'Mixed' 'Other']

Level-1 agents in the dataset:
['Human-directed' 'Wild' 'Mixed' 'Other']

Level-1 agents in the dataset:
['Human-directed' 'Wild' 'Mixed' 'Other']

Level-1 agents in the dataset:
['Human-directed' 'Wild' 'Mixed' 'Other']



In [14]:
## To adjust disturbed area
ADJUST = True
def area_adjust(_rec_agent_area_year):
    _rec_agent_area_year['maparea'] = _rec_agent_area_year['area'] # backup the original area which the mapped area
    agent_list = _rec_agent_area_year['agent'].unique()
    for agent_name in agent_list:
        adj_area, adj_area_std = get_area_adjust_factor(agent_name)
        _rec_agent_area_year.loc[(rec_agent_area_year['agent'] ==agent_name), 'area'] = \
            _rec_agent_area_year.loc[(_rec_agent_area_year['agent'] ==agent_name), 'area']*adj_area
    return _rec_agent_area_year

if ADJUST:
    rec_agent_area_year = area_adjust(rec_agent_area_year)
    nca1_agent_area_year = area_adjust(nca1_agent_area_year)
    nca2_agent_area_year = area_adjust(nca2_agent_area_year)
    nca3_agent_area_year = area_adjust(nca3_agent_area_year)
    nca4_agent_area_year = area_adjust(nca4_agent_area_year)
    nca5_agent_area_year = area_adjust(nca5_agent_area_year)
    nca6_agent_area_year = area_adjust(nca6_agent_area_year)
    nca7_agent_area_year = area_adjust(nca7_agent_area_year)

In [15]:
## bars

# define color code
level0_order = ["Disturbance"]
# white
level0_colors = ["#FFFFFF"]
# red hex for level0
level0_colors = ["#DB0000"]
# black
level0_colors = ["#000000"]

# blind safe color 
level1_order = ["Human-directed", "Wild", "Mixed"]
level1_colors = ["#CC79A7", "#009E73", "#D55E00"]

level2_order = [
    "Logging",
    "Construction",
    "Agricultural disturbance",
    "Stress",
    "Wind/geohazard",
    "Water disturbance",
    "Fire",
]
level2_order = [
    "Stress",
    "Logging",
    "Wind/geohazard",
    "Construction",
    "Fire",
    "Agricultural disturbance",
    "Water disturbance",
]
# blind safe color
level2_colors = [
    "#009E73",
    "#CC79A7",
    "#F0E442",
    "#E69F00",
    "#0072B2",
    "#56B4E9",
    "#D55E00",
]
# normal color bright
level2_colors = [
    "#BCD1C1",
    "#F4B2B2",
    "#FFFFB2",
    "#D1C8C0",
    "#CEBFD4",
    "#B2C3DF",
    "#FAD5B3",
]
# normal color dark
level2_colors = [
    "#226633", "Logging",
    "#DB0000", "Construction",
    "#FFFF00", "Agricultural disturbance",
    "#664830", "Stress",
    "#5D2C70", "Wind/geohazard",
    "#003994", "Water disturbance",
    "#F07605", "Fire",
]

# normal color dark
level2_colors = [
    "#664830",
    "#226633",
    "#5D2C70",
    "#DB0000",
    "#F07605",
    "#FFFF00",
    "#003994",
]



dir_figures = '/home/shq19004/Code/CONUSDisturbance/Analysis/Python2CreateFigure/figures'
        
total_area_conus = rec_agent_area_year['maparea'].sum() # include "other"

area_report = pandas.DataFrame()



# pie chart to show total area disturbed
for i, sub_rec in enumerate([rec_agent_area_year, nca1_agent_area_year, nca2_agent_area_year, nca3_agent_area_year, nca4_agent_area_year, nca5_agent_area_year, nca6_agent_area_year, nca7_agent_area_year]):

        _rec_agent_area_total = sub_rec.copy()

        # conus
        # 1: northwest
        # 2: midwest
        # 3: southeast
        # 4: northern great plains
        # 5: southern great plains
        # 6: northwest
        # 7: southwest
        if i == 0:
            regionname = 'CONUS'
        elif i == 1:
            regionname = 'Northeast'
        elif i == 2:
            regionname = 'Midwest'
        elif i == 3:
            regionname = 'Southeast'
        elif i == 4:
            regionname = 'Northern great plains'
        elif i == 5:
            regionname = 'Southern great plains'
        elif i == 6:
            regionname = 'Northwest'
        elif i == 7:
            regionname = 'Southwest'
        print('region: ', regionname)
        
        ## Level-0 disturbance and non-disturbance information
        total_maparea_disturbed = _rec_agent_area_total[_rec_agent_area_total['agent'].isin(level2_order)]['maparea'].sum() # exclude "other"
        # adjust the area   
        adj_area, adj_area_std = get_area_adjust_factor('Disturbance', level = 0) # level 0 is disturbance and non-disturbance
        # total mapped area for the agent
        ci = 1.96*adj_area_std*total_maparea_disturbed/1E6 # to mha
        total_area_disturbed = total_maparea_disturbed*adj_area/1E6 # to mha
        # print the adjusted area with uncertainty
        # two decimal
        print('total disturbance area of the region (adjusted) in mha with 95% CI: ', round(total_area_disturbed, 2), '+-', round(ci, 2))
        # append to the list
        report_region = {'Region': regionname, 'All disturbance area': total_area_disturbed, 'All disturbance ci': ci}
 
        ## Level-2 disturbance information
        # only maintian the agents
        _rec_agent_area_total = _rec_agent_area_total[_rec_agent_area_total['agent'].isin(level2_order)]
        _rec_agent_area_total_agent = _rec_agent_area_total.groupby(['agent'])['area'].sum().reset_index()

        # sort the dataframe by agent name
        # Order list
        _rec_agent_area_total_agent['agent'] = pandas.Categorical(_rec_agent_area_total_agent['agent'], categories=level2_order, ordered=True)
        _rec_agent_area_total_agent = _rec_agent_area_total_agent.sort_values('agent')
        # display the data by bar by plotly
        fig = go.Figure()
        fig.add_trace(go.Bar(x=_rec_agent_area_total_agent['agent'], y=_rec_agent_area_total_agent['area']/1E6, marker_color=level2_colors))

        # # estimate the error bar at 95% CI, according to the mapped area and the standard error obtained by validation
        # ci_area = []
        # for j, agent in enumerate(level2_order):
        #     std_area_p = adj_area_std_error[j]
        #     # total mapped area for the agent
        #     ci = 1.96*std_area_p*total_maparea*(total_maparea/total_maparea_conus)/1E6 # to mha
        #     # append to the list
        #     ci_area.append(ci)
        # # add error bar
        # fig.add_trace(go.Scatter(x=_rec_agent_area_total_agent['agent'], y=_rec_agent_area_total_agent['area']/1E6, 
        #                          error_y=dict(type='data', array=ci_area, visible=True), mode='markers', 
        #                          marker=dict(color='black', size=1)))
        
        # adjust the error if the minimum value is less than 0
        ci_area_upper = []
        ci_area_lower = []
        for j, agent in enumerate(level2_order):
            adj_area, adj_area_std = get_area_adjust_factor(agent)
            real_area = _rec_agent_area_total.loc[(_rec_agent_area_total['agent'] == agent), 'area'].sum()  # in which the area has been adjusted
            map_area = _rec_agent_area_total.loc[(_rec_agent_area_total['agent'] == agent), 'maparea'].sum() # in which the area has been adjusted
            # total mapped area for the agent
            ci = 1.96*adj_area_std*map_area/1E6 # to mha
            # append to the list
            ci_area_upper.append(ci)
            if real_area - ci <= 0:
                ci_area_lower.append(real_area) # make the lower bound not less than 0
            else:
                ci_area_lower.append(ci)
            
            # print the adjusted area with uncertainty
            # two decimal
            print('total disturbance area of {:} (adjusted) in mha with 95% CI: '.format(agent), round(real_area/1E6, 2), '+-', round(ci, 2))
            # append to the list
            report_region['{:} area'.format(agent)] = real_area/1E6
            report_region['{:} ci'.format(agent)] = ci
        # append to the dataframe
        report_region = pandas.DataFrame(report_region, index=[0])
        # combine the data into area_report together
        area_report = pandas.concat([area_report, report_region], ignore_index=True)

        
        # add error bar
        fig.add_trace(go.Scatter(x=_rec_agent_area_total_agent['agent'], y=_rec_agent_area_total_agent['area']/1E6, 
                                error_y=dict(
                                    type='data',
                                    symmetric=False,
                                    array=ci_area_upper,
                                    arrayminus=ci_area_lower),
                                 mode='markers', 
                                 marker=dict(color='black', size=1)))
        # adust the size of error bar
        fig.update_traces(error_y_width=2)
        fig.update_traces(error_y_thickness=0.5)
        
        # do not show the legend
        fig.update_layout(showlegend=False)

        # closer to each other
        fig.update_layout(bargap=0.01)
        # do not show the xstick
        fig.update_xaxes(showticklabels=False)
        fig.update_xaxes(
            ticks="inside",
            tickson="boundaries",
            ticklen=0
        )
        # remove the x axis
        fig.update_xaxes(showline=True, linewidth=0, linecolor='black')
        
        # keep two decimal in int and 0.1f to make all them same size to display
        # set up the mininum of y axis
        # fig.update_layout(yaxis_range=[0, (_rec_agent_area_total_agent['area']/1E6 + ci_area_upper).max()])
        # change the size as to 4, 4
        fig.update_layout(width=100, height=90)
        if _rec_agent_area_total_agent['area'].max()/1E6 > 10:
            fig.update_layout(yaxis_tickformat = 'd')
        else:
            fig.update_layout(yaxis_tickformat = '.1f')
        # change as to simple-white
        fig.update_layout(template='simple_white')
        # change font as to 12 and arial
        fig.update_layout(font=dict(family="Arial", size=12))
        # fig.update_layout(width=1000, height=800)
        # remove the margin of the border
        fig.update_layout(margin=dict(l=0, r=0, t=0, b=0))
        # setup the background color as to transparent
        fig.update_layout(plot_bgcolor='rgba(0,0,0,0)')
        fig.update_layout(paper_bgcolor='rgba(0,0,0,0)')
        # show the figure
        fig.show()

        # save as png with transparent background
        fig.write_image(os.path.join(dir_figures, 'figure_area_hist_{:}.png'.format(i)), scale=2)
    
# save the area report
# switch the columns with the rows
# area_report.to_csv(os.path.join(dir_figures, 'disturbance_area_report.csv'), index=False)

region:  CONUS
total disturbance area of the region (adjusted) in mha with 95% CI:  178.8 +- 7.74
total disturbance area of Stress (adjusted) in mha with 95% CI:  24.38 +- 3.13
total disturbance area of Logging (adjusted) in mha with 95% CI:  58.85 +- 6.15
total disturbance area of Wind/geohazard (adjusted) in mha with 95% CI:  2.6 +- 2.04
total disturbance area of Construction (adjusted) in mha with 95% CI:  13.73 +- 1.99
total disturbance area of Fire (adjusted) in mha with 95% CI:  15.4 +- 1.27
total disturbance area of Agricultural disturbance (adjusted) in mha with 95% CI:  43.0 +- 5.81
total disturbance area of Water disturbance (adjusted) in mha with 95% CI:  20.54 +- 2.74


region:  Northeast
total disturbance area of the region (adjusted) in mha with 95% CI:  3.5 +- 0.15
total disturbance area of Stress (adjusted) in mha with 95% CI:  0.04 +- 0.01
total disturbance area of Logging (adjusted) in mha with 95% CI:  1.89 +- 0.2
total disturbance area of Wind/geohazard (adjusted) in mha with 95% CI:  0.04 +- 0.03
total disturbance area of Construction (adjusted) in mha with 95% CI:  0.92 +- 0.13
total disturbance area of Fire (adjusted) in mha with 95% CI:  0.02 +- 0.0
total disturbance area of Agricultural disturbance (adjusted) in mha with 95% CI:  0.27 +- 0.04
total disturbance area of Water disturbance (adjusted) in mha with 95% CI:  0.32 +- 0.04


region:  Midwest
total disturbance area of the region (adjusted) in mha with 95% CI:  7.56 +- 0.33
total disturbance area of Stress (adjusted) in mha with 95% CI:  0.02 +- 0.0
total disturbance area of Logging (adjusted) in mha with 95% CI:  2.04 +- 0.21
total disturbance area of Wind/geohazard (adjusted) in mha with 95% CI:  0.03 +- 0.02
total disturbance area of Construction (adjusted) in mha with 95% CI:  1.62 +- 0.23
total disturbance area of Fire (adjusted) in mha with 95% CI:  0.05 +- 0.0
total disturbance area of Agricultural disturbance (adjusted) in mha with 95% CI:  2.44 +- 0.33
total disturbance area of Water disturbance (adjusted) in mha with 95% CI:  1.12 +- 0.15


region:  Southeast
total disturbance area of the region (adjusted) in mha with 95% CI:  61.29 +- 2.65
total disturbance area of Stress (adjusted) in mha with 95% CI:  0.12 +- 0.01
total disturbance area of Logging (adjusted) in mha with 95% CI:  41.3 +- 4.32
total disturbance area of Wind/geohazard (adjusted) in mha with 95% CI:  2.39 +- 1.87
total disturbance area of Construction (adjusted) in mha with 95% CI:  5.76 +- 0.84
total disturbance area of Fire (adjusted) in mha with 95% CI:  0.97 +- 0.08
total disturbance area of Agricultural disturbance (adjusted) in mha with 95% CI:  4.0 +- 0.54
total disturbance area of Water disturbance (adjusted) in mha with 95% CI:  5.96 +- 0.79


region:  Northern great plains
total disturbance area of the region (adjusted) in mha with 95% CI:  18.06 +- 0.78
total disturbance area of Stress (adjusted) in mha with 95% CI:  3.55 +- 0.45
total disturbance area of Logging (adjusted) in mha with 95% CI:  1.13 +- 0.12
total disturbance area of Wind/geohazard (adjusted) in mha with 95% CI:  0.02 +- 0.01
total disturbance area of Construction (adjusted) in mha with 95% CI:  0.27 +- 0.04
total disturbance area of Fire (adjusted) in mha with 95% CI:  1.7 +- 0.14
total disturbance area of Agricultural disturbance (adjusted) in mha with 95% CI:  7.61 +- 1.03
total disturbance area of Water disturbance (adjusted) in mha with 95% CI:  3.79 +- 0.51


region:  Southern great plains
total disturbance area of the region (adjusted) in mha with 95% CI:  37.79 +- 1.64
total disturbance area of Stress (adjusted) in mha with 95% CI:  11.54 +- 1.48
total disturbance area of Logging (adjusted) in mha with 95% CI:  4.88 +- 0.51
total disturbance area of Wind/geohazard (adjusted) in mha with 95% CI:  0.08 +- 0.06
total disturbance area of Construction (adjusted) in mha with 95% CI:  2.92 +- 0.42
total disturbance area of Fire (adjusted) in mha with 95% CI:  1.19 +- 0.1
total disturbance area of Agricultural disturbance (adjusted) in mha with 95% CI:  14.2 +- 1.92
total disturbance area of Water disturbance (adjusted) in mha with 95% CI:  3.56 +- 0.47


region:  Northwest
total disturbance area of the region (adjusted) in mha with 95% CI:  14.74 +- 0.64
total disturbance area of Stress (adjusted) in mha with 95% CI:  0.27 +- 0.03
total disturbance area of Logging (adjusted) in mha with 95% CI:  5.44 +- 0.57
total disturbance area of Wind/geohazard (adjusted) in mha with 95% CI:  0.02 +- 0.02
total disturbance area of Construction (adjusted) in mha with 95% CI:  0.33 +- 0.05
total disturbance area of Fire (adjusted) in mha with 95% CI:  3.3 +- 0.27
total disturbance area of Agricultural disturbance (adjusted) in mha with 95% CI:  3.4 +- 0.46
total disturbance area of Water disturbance (adjusted) in mha with 95% CI:  1.28 +- 0.17


region:  Southwest
total disturbance area of the region (adjusted) in mha with 95% CI:  35.86 +- 1.55
total disturbance area of Stress (adjusted) in mha with 95% CI:  8.84 +- 1.13
total disturbance area of Logging (adjusted) in mha with 95% CI:  2.18 +- 0.23
total disturbance area of Wind/geohazard (adjusted) in mha with 95% CI:  0.03 +- 0.02
total disturbance area of Construction (adjusted) in mha with 95% CI:  1.91 +- 0.28
total disturbance area of Fire (adjusted) in mha with 95% CI:  8.18 +- 0.68
total disturbance area of Agricultural disturbance (adjusted) in mha with 95% CI:  11.07 +- 1.5
total disturbance area of Water disturbance (adjusted) in mha with 95% CI:  4.5 +- 0.6


In [17]:
# show the annual disturbance area for each agent and fit its trend using sen's slope for conus-wide
# conus
from scipy import stats  # for sen's slope
import pymannkendall as mk

_ci = 0.95
# change the agent name
rec_agent_area_year.loc[
    (rec_agent_area_year["agent"] == "forest_management"), "agent"
] = "Logging"
rec_agent_area_year.loc[(rec_agent_area_year["agent"] == "construction"), "agent"] = (
    "Construction"
)
rec_agent_area_year.loc[(rec_agent_area_year["agent"] == "stress"), "agent"] = "Stress"
rec_agent_area_year.loc[(rec_agent_area_year["agent"] == "natural_hazard"), "agent"] = (
    "Wind/geohazard"
)
rec_agent_area_year.loc[(rec_agent_area_year["agent"] == "water_dynamic"), "agent"] = (
    "Water disturbance"
)
rec_agent_area_year.loc[(rec_agent_area_year["agent"] == "fire"), "agent"] = "Fire"
rec_agent_area_year.loc[
    (rec_agent_area_year["agent"] == "agriculture_activity"), "agent"
] = "Agricultural disturbance"


def init_figure(rows=4, cols=2):
    # plot the scatter plot by plotly
    # fig = make_subplots(rows=rows, cols=cols, vertical_spacing=0.02, horizontal_spacing= 0.04)
    fig = make_subplots(rows=rows, cols=cols, vertical_spacing=0.04, horizontal_spacing= 0.08)
    # change as to simple-white
    fig.update_layout(template="simple_white")
    # change font as to 12 and arial
    fig.update_layout(font=dict(family="Arial", size=12))
    # remove the margin of the border
    fig.update_layout(margin=dict(l=0, r=0, t=0, b=0))
    # fig.update_layout(width=350, height=200)
    # fig.update_layout(width=300, height=600)
    fig.update_layout(width=650, height=550)
    # setup the background color as to transparent
    # fig.update_layout(plot_bgcolor='rgba(0,0,0,0)')
    # fig.update_layout(paper_bgcolor='rgba(0,0,0,0)')
    # share the x axis and y axis
    # fig.update_layout(plot_bgcolor='black', paper_bgcolor='black')
    # change font to Arial, size 12, and color to white
    # fig.update_layout(font=dict(family="Arial", size=12, color='white'))
    
    return fig


def save_figure(fig, name):
    # save as png with transparent background
    dir_figures = (
        "/home/shq19004/Code/CONUSDisturbance/Analysis/Python2CreateFigure/figures"
    )
    fig.write_image(
        os.path.join(dir_figures, "figure_conus_area_trend_{:}.svg".format(name)),
        # scale=2,
    )


def plot_lines_sns(agent, level, rec_agent_area_year, color_code, fig, plot_row=1, plot_col=1):

    # select the legend names according to inputting agents
    rec_agent_area_year = rec_agent_area_year[rec_agent_area_year.agent.isin([agent])]

    # sum the area and maparea according to the agent
    rec_agent_area_year = (
        rec_agent_area_year.groupby(["agent", "year"])[["area", "maparea"]]
        .sum()
        .reset_index()
    )

    rec_agent_area_year["area"] = rec_agent_area_year["area"] / 1e6  # ha to Mha

    # plot the scatter plot by plotly
    #  fig = init_figure()
    fig.append_trace(
        go.Scatter(
            x=rec_agent_area_year["year"],
            y=rec_agent_area_year["area"],
            mode="markers",
            marker=dict(color=color_code, size=6),
            name=agent,
            showlegend=False,
        ),
        row=plot_row,
        col=plot_col,
    )

    # add error bar
    # get the ci range
    ci_area_upper, ci_area_lower = get_confident_interval(
        rec_agent_area_year, agent, level
    )
    fig.append_trace(
        go.Scatter(
            showlegend=False,
            x=rec_agent_area_year["year"],
            y=rec_agent_area_year["area"],
            error_y=dict(
                type="data",
                symmetric=False,
                array=ci_area_upper,
                arrayminus=ci_area_lower,
                thickness=1  # Removes the horizontal cap on the error bars
            ),
            mode="markers",
            marker=dict(color="gray", size=1),
        ),
        row=plot_row,
        col=plot_col,
    )

    # get the x and y with arrray
    _x = rec_agent_area_year["year"].values
    _y = rec_agent_area_year["area"].values

    # add sen's slope
    # get the sen's slope
    # see https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.theilslopes.html
    slope, intercept, low_slope, high_slope = stats.theilslopes(
        _y, _x, _ci, method="separate"
    )
    # https://pypi.org/project/pymannkendall/#history
    # https://github.com/mmhs013/pyMannKendall/blob/master/Examples/Example_pyMannKendall.ipynb
    # Original Mann-Kendall test for trend
    trend, h, p, z, Tau, s, var_s, _slope, _intercept = mk.original_test(_y, alpha = _ci)  
    # the two packages give the same result of slope
    
    # print
    slope_values = "Trend = {:.0f} [{:.0f}, {:.0f}] ha/year".format(
        slope * 1e6, low_slope * 1e6, high_slope * 1e6
    )
    print(agent, slope_values)
    p_values = "p-value = {:.05f}".format(p)
    print(agent, p_values)
    

    fig.append_trace(
        go.Scatter(
            x=_x,
            y=intercept + slope * _x,
            mode="lines",
            line=dict(color=color_code, width=2),
            name=slope_values,
            legendgroup=agent,
        ),
        row=plot_row,
        col=plot_col,
    )
    # add confidence interval with low_slope and high_slope by shadow color
    # convert the hex to rgb
    _color_code = color_code.lstrip("#")
    _color_code = tuple(int(_color_code[i : i + 2], 16) for i in (0, 2, 4))
    fillcolor = "rgba" + str(_color_code)[0:-1] + ", 0.4)"

    # show the upper and lower bound around the trend line at the mean of _x
    _x_mean = np.mean(_x)
    _y_mean = intercept + slope * _x_mean

    low_intercept = _y_mean - low_slope * _x_mean
    high_intercept = _y_mean - high_slope * _x_mean

    fig.append_trace(
        go.Scatter(
            x=_x,
            y=low_intercept + low_slope * _x,
            mode="lines",
            line=dict(color=color_code, width=0),
            name="Low",
            fillcolor=fillcolor,
            showlegend=False,
        ),
        row=plot_row,
        col=plot_col,
    )
    fig.append_trace(
        go.Scatter(
            x=_x,
            y=high_intercept + high_slope * _x,
            mode="lines",
            line=dict(color=color_code, width=0),
            name="High",
            fill="tonexty",
            fillcolor=fillcolor,
            showlegend=False,
        ),
        row=plot_row,
        col=plot_col,
    )
    # do not show the line of lower and upper bound

    # add y axis label
    # fig.update_yaxes(title_text="Area (Mha)",
    #     row=plot_row,
    #     col=plot_col,)
    # move the legend to the upper left
    # fig.update_layout(
    #     legend=dict(
    #         x=0.0,
    #         y=1.1,
    #         bgcolor="rgba(0,0,0,0)",
    #         xanchor="left",
    #         yanchor="top",
    #     ),
    # )
    # display years
    # limite the x axis into the years
    fig.update_xaxes(range=[1988 - 1, 2022 + 1],
        row=plot_row,
        col=plot_col,)
    if plot_row == 1:
        fig.update_xaxes(tickvals=[1990, 2000, 2010, 2020],),
    else:
        # do not show the xstick
        fig.update_xaxes(showticklabels=False)
    # y axis with two decimal
    # if level == 0:
    #    fig.update_layout(yaxis_tickformat="d")
    # else:
    #     fig.update_layout(yaxis_tickformat=".1f")
    
    # hide legend
    fig.update_layout(showlegend=False)
    

def get_confident_interval(rec_agent_area_year, agent_name, level):
    adj_area, adj_area_std = get_area_adjust_factor(agent_name, level=level)
    # get years
    years = rec_agent_area_year["year"].unique()

    ci_area_upper = []
    ci_area_lower = []
    for j, agent in enumerate(years):
        total_agent_maparea_conus_annual = rec_agent_area_year[
            (rec_agent_area_year["agent"] == agent_name)
            & (rec_agent_area_year["year"] == agent)
        ]["maparea"].values[0]
        real_area = rec_agent_area_year[
            (rec_agent_area_year["agent"] == agent_name)
            & (rec_agent_area_year["year"] == agent)
        ]["area"].values[0]
        # total mapped area for the agent
        ci = 1.96 * adj_area_std * total_agent_maparea_conus_annual / 1e6  # to mha
        # append to the list
        ci_area_upper.append(ci)
        if real_area - ci <= 0:
            ci_area_lower.append(real_area)  # make the lower bound not less than 0
        else:
            ci_area_lower.append(ci)
    return ci_area_upper, ci_area_lower



# show those plot into one figure
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# plot the trend for each agent level-2
fig = init_figure()
iplot = 0
for agent in level0_order:
    _rec_agent_area_year = rec_agent_area_year.copy()
    # updathe agent name as disturbance if it is not other
    _rec_agent_area_year.loc[(_rec_agent_area_year["agent"] != "Other"), "agent"] = (
        "Disturbance"
    )
    # get row and col by the iplot
    row = 1
    col = 1
    plot_lines_sns(
        agent,
        0,
        _rec_agent_area_year,
        level0_colors[level0_order.index(agent)],
        fig,
        row,
        col
    )
    iplot = iplot + 1

# plot the trend for each agent level-1
for agent in ["Human-directed", "Wild"]: # indicate the human-directed and wild disturbance, repsectively
    # get the row and col by the iplot
    # row = iplot + 1
    row = 1
    col = 1
    _rec_agent_area_year = rec_agent_area_year.copy()
    # give the agent_level1 to agent to make it functional
    _rec_agent_area_year["agent"] = _rec_agent_area_year["agent_level1"]
    plot_lines_sns(
        agent, 1, _rec_agent_area_year, level1_colors[level1_order.index(agent)], fig
    )
    
# plot the trend for each agent level-2
for agent in level2_order:
    # get the row and col by the iplot
    row = iplot + 1
    col = 1
    row = iplot // 2 + 1
    col = iplot % 2 + 1
    plot_lines_sns(
        agent,
        2,
        rec_agent_area_year.copy(),
        level2_colors[level2_order.index(agent)],
        fig,
        row,
        col
    )
    iplot = iplot + 1

fig.show()
save_figure(fig, 'conus_area_trend')


Disturbance Trend = -47733 [-81437, -20424] ha/year
Disturbance p-value = 0.00377
Human-directed Trend = -59208 [-80901, -43025] ha/year
Human-directed p-value = 0.00000
Wild Trend = 20312 [1966, 35654] ha/year
Wild p-value = 0.02484
Stress Trend = 610 [-3956, 10643] ha/year
Stress p-value = 0.82025
Logging Trend = -8143 [-13861, -2966] ha/year
Logging p-value = 0.00196
Wind/geohazard Trend = 950 [-20, 1639] ha/year
Wind/geohazard p-value = 0.06085
Construction Trend = -6720 [-9296, -3861] ha/year
Construction p-value = 0.00006
Fire Trend = 11237 [3899, 23541] ha/year
Fire p-value = 0.00697
Agricultural disturbance Trend = -43523 [-62841, -26338] ha/year
Agricultural disturbance p-value = 0.00000
Water disturbance Trend = -6202 [-10120, -1891] ha/year
Water disturbance p-value = 0.00587
